In [54]:
!pip install transformers
!pip install happytransformer
!pip install vaderSentiment
!pip install textblob
!pip install summa
!pip install language-check

You should consider upgrading via the 'C:\Users\rietesh.amminabhavi\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\rietesh.amminabhavi\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\rietesh.amminabhavi\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\rietesh.amminabhavi\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [29 lines of output]
      running bdist_wheel
      The [wheel] section is deprecated. Use [bdist_wheel] instead.
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-3.9
      creating build\lib.win-amd64-3.9\pycld2
      copying pycld2\__init__.py -> build\lib.win-amd64-3.9\pycld2
      running build_ext
      building 'pycld2._pycld2' extension
      creating build\temp.win-amd64-3.9
      creating build\temp.win-amd64-3.9\Release
      creating build\temp.win-amd64-3.9\Release\Users
      creating build\temp.win-amd64-3.9\Release\Users\rietesh.amminabhavi
      creating build\temp.win-amd64-3.9\Release\Users\rietesh.amminabhavi\AppData
      creating build\temp.win-amd64-3.9\Release\Users\rietesh.amminabhavi\AppData\Local
      creating build\temp.win-amd64-3.9\Release\Users\rietesh.amminabhavi\AppData\Local\Temp
  


     -------------------------------------- 41.4/41.4 MB 585.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pycld2
Failed to build pycld2
  Running setup.py install for pycld2: started
  Running setup.py install for pycld2: finished with status 'error'


In [2]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from summa import keywords
import language_tool_python
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

data = pd.read_csv('./chrome_reviews.csv')
data.shape

(7204, 10)

In [3]:
# Task 1
import re
string = '{"orders":[{"id":1},{"id":2},{"id":3},{"id":4},{"id":5},{"id":6},{"id":7},{"id":8},{"id":9},{"id":10},{"id":11},{"id":648},{"id":649},{"id":650},{"id":651},{"id":652},{"id":653}],"errors":[{"code":3,"message":"[PHP Warning #2] count(): Parameter must be an array or an object that implements Countable (153)"}]}'
print(re.findall(r'(?<=:)\w+', string))


['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '648', '649', '650', '651', '652', '653', '3']


In [59]:
text = data['Text'].astype(str).values.tolist()
sentiments = SentimentIntensityAnalyzer()
vader = []
blob = []
for i in text:
  sentiment_dict = sentiments.polarity_scores(i)
  vader.append(sentiment_dict)
  blob.append(TextBlob(i).sentiment.polarity)

results = pd.DataFrame(vader)
results['text'] = text
results['star'] = data['Star'].values
results['blob'] = blob
results

,neg,neu,pos,compound,text,star,blob
0,0.000,1.000,0.000,0.0000,This is very helpfull aap.,5,0.200000
1,0.000,0.000,1.000,0.4404,Good,3,0.700000
2,0.000,1.000,0.000,0.0000,Not able to update. Neither able to uninstall.,1,0.125000
3,0.000,0.263,0.737,0.4215,Nice app,4,0.600000
4,0.487,0.513,0.000,-0.2263,Many unwanted ads,1,0.500000
...,...,...,...,...,...,...,...
7199,0.000,1.000,0.000,0.0000,Bagusss..,5,0.000000
7200,0.515,0.294,0.191,-0.4939,Bad version 😔,1,-0.700000
7201,0.054,0.669,0.277,0.8395,One thing that I have to say I can't spelled t...,5,0.850000
7202,0.000,0.000,1.000,0.5719,Excellent,5,1.000000


In [60]:
results['sent_res'] = results.apply(lambda x: ['neg','neu','pos'][np.argmax([x['neg'],x['neu'], x['pos']])], axis=1)
results['star_res'] = results['star'].apply(lambda x: 'pos' if x>3 else ('neu' if x==3 else 'neg'))
pos_neg = results[(results['sent_res']=='pos') & (results['star_res']=='neg')][['text','star','sent_res']].reset_index()
neg_pos = results[(results['sent_res']=='neg') & (results['star_res']=='pos')][['text','star','sent_res']].reset_index()
answer = pd.concat([neg_pos,pos_neg])
answer

,index,text,star,sent_res
0,58,Not good!,4,neg
1,386,Good but bad pictures,5,neg
2,455,Bad,5,neg
3,579,Not good,5,neg
4,845,No leer esto,5,neg
...,...,...,...,...
152,7041,Good,1,pos
153,7042,Good app,1,pos
154,7047,Its good,1,pos
155,7060,👌👌,1,pos


## Ranking Dataset

In [61]:
df = pd.read_csv('browser_rankings_data.csv')
df.head()

,ID,Keyword,Rank,Country,Language,Date,App ID,Date of Last Description Change,Short Description,Long Description
0,1,browser,2.0,in,en,2020-12-16,net.fast.web.browser,2020-12-18,"The small, fastest, secure Web Browser, best m...","One of the smallest, fastest, secure Web Brows..."
1,2,browser,3.0,in,en,2020-12-16,com.android.chrome,2020-12-18,"Fast, simple, and secure. Google Chrome browse...","Google Chrome is a fast, easy to use, and secu..."
2,3,browser,4.0,in,en,2020-12-16,com.opera.browser,2021-02-27,"Fast, safe web browser: free VPN, Ad blocker, ...","Fast, safe and private, introducing the latest..."
3,3,browser,4.0,in,en,2020-12-16,com.opera.browser,2020-12-18,"Fast, safe web browser: free VPN, Ad blocker, ...","Opera is a fast, safe web browser for your And..."
4,4,browser,1.0,in,en,2020-12-16,com.cloudmosa.puffinTV,2021-02-07,Puffin TV Browser - One App to Watch All Video...,Puffin TV Browser is a special edition of Puff...


In [63]:
df['vader_polarity_short'] = df['Short Description'].apply(lambda x: sentiments.polarity_scores(x)['compound'])
df['blob_polarity_short'] = df['Short Description'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['vader_polarity_long'] = df['Long Description'].apply(lambda x: sentiments.polarity_scores(x)['compound'])
df['blob_polarity_long'] = df['Long Description'].apply(lambda x: TextBlob(x).sentiment.polarity)
df = df.dropna()

for i in ['vader_polarity_short','blob_polarity_short','vader_polarity_long','blob_polarity_long']:
    corr, pvalue = pearsonr(df['Rank'].values, df[i].values)
    print('Correlation Btw {} and Rank is: {} with pvalue {}'.format(i,corr,pvalue))

Correlation Btw vader_polarity_short and Rank is: -0.2819732966246105 with pvalue 7.47976531254696e-57
Correlation Btw blob_polarity_short and Rank is: -0.18960040347965534 with pvalue 4.438187052396157e-26
Correlation Btw vader_polarity_long and Rank is: 0.17994716646681863 with pvalue 1.3004533176745527e-23
Correlation Btw blob_polarity_long and Rank is: 0.028470192724569517 with pvalue 0.11595242983445699


In [80]:
def if_in_first_10(desc, words):
    desc = desc.split()[:15]
    score = 0
    for i in words:
        if i in desc:
            score += 1
    return score

df['keywords'] = df['Long Description'].apply(lambda x: [i[0] for i in keywords.keywords(x, scores=True)[0:3]])
df['keyword_score'] = df.apply(lambda row: if_in_first_10(row['Long Description'], row['keywords']), axis=1)

corr, pvalue = pearsonr(df['Rank'].values, df['keyword_score'].values)
print('Correlation Btw {} and Rank is: {} with pvalue {}'.format('keyword_score',corr,pvalue))

Correlation Btw keyword_score and Rank is: 0.30440746952997183 with pvalue 2.0595631649171097e-66


# Grammatical Correction

In [3]:
review = pd.read_csv('review_data.csv')
review.shape

(30000, 4)

In [88]:
from happytransformer import  HappyTextToText

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

review['grammar'] = 'grammar: '+review['text'].astype(str)
review['error'] = review.apply(lambda row: True if row['text'] == happy_tt.generate_text(row['grammar']).text else False, axis=1)

review

In [5]:
# tool = language_tool_python.LanguageTool('en-US')

# def check_filter(string):
#     methods = tool.check(string)
#     value = len(methods)
#     return min(value,1)

# review['Is_Error'] = review['text'].apply(check_filter)

In [9]:
text = review.text.values.tolist()
len(text)

30000